# Analysis of RF heating (low $q_x$)
This program is devoted to the analysis of RF heating simulations, done with MD program. Initially, first simulation were carried out in 2022, 30 of september (see `20220930`) but the aspect ratio of the potential was not close enough to 1. The problem was that the effect of the axial potential over radial direction was not properly considered. This implies to properly compute the beta parameter in a recursive way. The computation of the right beta is done in another Python program called `2021_Mathieu_parameters`. The data in `20220930` is viable, but do not have the symmetrical aspect ratio.
The low q data used in the RF heating article (2023) are ultimately analysed with this program (from $q=0.2$ to $q=0.6$). Data is stored on Rivendel in the following folder `Rivendel/Simulations/20221017`, but you will have to manually add the $q_x=0.6$ from `20220617`. They are first pre-processed with a python program, `20220617_extractdatatonpz`, converting the raw data into `.npz` archives. Then the `.npz` archives are opened in this program and analysed.

This analysis program first opens the `.npz`, proposes to plot some temperature curves. A temporal window average is carried out, then the heating rate is numerically computed. Analytical cooling power is computed, along with heating rate from the spontaneous emission. Those computations are compared in a figure.

Keep in mind the analysis of low $q_x$ from $q_x=0.2$ to $q_x=0.6$ is done with another program called `20220617_Heating_rate_high_q-ForArticle`.

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate
import matplotlib.ticker as ticker

# %matplotlib qt

In [6]:
%run functions
matplotlib.rcParams.update({'font.size': 25})
cm = pylab.get_cmap('tab10')
cm2 = pylab.get_cmap('Set1')
# cm3 = pylab.get_cmap('turbo')
cm3j = pylab.get_cmap('jet')
cm4 = pylab.get_cmap('viridis')

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6
tauRF = 1/(Omega/2/pi)

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Open data stored as `.npz`
If you don't have the `.npz` archives please first convert the data using `20220617_extractdatatonpz` python program.

In [8]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

# Use 20220617_extractdatatonpz before

# Data is in the file /home/adrien/RemoteFS/Rivendel/Simulations/20221017

time       = []
T_aux_avg  = []
r2_v2_rlim = []
alpha       = []

for i,j in enumerate(file_path):
    print(j)
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])
        alpha.append(data['alpha'])
        
###########
# You should have
# Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
# Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
# Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz
# Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC05_RF05.npz
# Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz

# Beware that after 05 it is 08 it is fine

K:/Universite_Simus/20230413/Time_and_temp_RFHEAT_N1024_alphavar_DC02_RF02.npz


In [9]:
# Checking aspect ratio
myRF = 0

for k in range(3):
    print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]

R = r2_v2_rlim[myRF][6][50]*1e-3
L = r2_v2_rlim[myRF][8][50]*1e-3
aws = 0
print(R,L)
print((R+(1.48*aws/2))/(L+1.48*aws/2))

density = 1024 / (4/3*pi*R**2*L)

190.70858957729456
188.5500129213134
186.40296076747995
0.00019070858957729456 0.00018640296076747996
1.0230985001101214


In [10]:
def func_lin(t,a,b):
    return a*t+b
def func_exp(x,a,b):
    return b*exp(-a*x)
def func_pow(t,A,B):
    return t**A * exp(B)

In [11]:
# Enters parameters
to_fit = 0
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
Urf = array([0, 0, 20.532292436697, 30.798438655045, 41.065, 51.33073109174249,0,0,6.1597])
q = array([0, 0, 0.2, 0.3, 0.4, 0.5, 0,0, 0.6])
Udc = array([0, 0, 1.3454211755646195, 3.062127421577093, 5.706, 8.837305210834145, 0,0, 13.08323245])
N_ions = array([1024,1024,1024,1024,1024])

# Time is corrected so it starts at 0
x  = time[to_fit]-1e-3
fs = 1/diff(x)[1]

In [12]:
# Display R/L aspect ratio
# Computed from Fortran MD program
for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    R = mean(r2_v2_rlim[cond][6][100:500])
    L = mean(r2_v2_rlim[cond][8][100:500])
    print(R/L)
    print(alpha[cond])
    print('')
address = file_path[0]
print(address)

1.0041567568299214
1.059416534255704

K:/Universite_Simus/20230413/Time_and_temp_RFHEAT_N1024_alphavar_DC02_RF02.npz


In [13]:
# Display parameters of simus
# in nice table

print('|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|',end='\n')
print('|---------|----|----|---------|---------|---------|-----|-----|',end='\n')

for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    
    print(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.2f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{beta_guess[my_try]:.3f}|',end='\n')

|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|
|---------|----|----|---------|---------|---------|-----|-----|


NameError: name 'omega_x_2' is not defined

In [160]:
# write parameters in a file
with open('data_parameters.txt','w') as f:
    f.write('2022 oct 21\n')
    f.write('Paramètres des simulations de chauffage RF\n')
    f.write('Simulations dans le dossier Rivendel/Simulations/20221017\n\n')
    
    f.write('|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|\n')
    f.write('|---------|----|----|---------|---------|---------|-----|-----|\n')

    for cond in range(len(file_path)):
        #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
        my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])

        f.write(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.2f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{beta_guess[my_try]:.3f}|\n')

# Some plots of temperature

In [14]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 0 #11 19 8
temp_to_end = 50

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]


figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax1.yaxis.set_major_locator(maj_loc)
ax1.yaxis.set_minor_locator(min_loc)

ax1.set_xlabel('t [ms]')
ax1.set_ylabel('T [K]')
# ax1.set_xlim(-0.1,4.3)
ax1.set_ylim(2e-3,300)
ax1.legend()
tight_layout()


# savefig(figname+'.eps',dpi=300)

file to analyze
> 0 K:/Universite_Simus/20230413/Time_and_temp_RFHEAT_N1024_alphavar_DC02_RF02.npz


H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel\eventloops.py:106: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


# From fluo-variations_optimal-temp.ipynb

# Functions used to describe ions
### From fluo-variations_optimal-temp.ipynb
I present the functions used to compute fluorescence and cooling power as a function of the temperature. See Foot chapter 7 and 9. The two basic functions are as follows.

- $\texttt{MB}(v)$ is the Maxwell-Boltzmann distribution for a given temperature. It provides the probability to find an atom with a given velocity in a gas with temperature T.

- $\texttt{pfl_dop}(v) = \rho_{ee}(v)$ is the atomic ray profile with Doppler effect. It provides the probability of excitation of a single oscillator given Rabi frequency, detuning, lambda and its velocity. It is considered equal to the excited population, i.e the proportion of excited atoms. This function is used with fixed Rabi frequency, detuning and lambda.

Those functions are used to compute the total fluorescence of the ion ensemble under laser cooling $F$, the cooling power of laser $G$, the heating induced by the spontaneous emission $G_{Hot}$.

- $\texttt{prob_fluo} = \texttt{pfl_dop}\times \texttt{MB}$ is the probability of excitation of an ion with velocity v in an ensemble at temperature $T$. The sum $F=\Gamma\sum_v \texttt{prob_fluo}\; dv$ is the fluorescence emitted by an ensemble of ions with given temperature $T$.

- $\texttt{cool_power} = \texttt{pfl_dop}\times \texttt{MB}\times kv$ is used to compute the cooling energy of laser $G = \sum_v \texttt{cool_power}\; \mathrm{d}v \times \;\hbar k \Gamma$ [J].

- $\texttt{hot_power} = \texttt{prob_fluo}$ is used to compute the heating induced by the spontaneous emission $G_{Hot} = \sum_v \texttt{hot_power}\; \mathrm{d}v \times \;\hbar^2 k^2 \Gamma/m$ [J].

You can multiply $G$ and $G_{Hot}$ by $1/(k_b\tau_{RF}$ to express those quantities in K/RF period. This transformation relies on the following formula : E = 3/2 k_bT.


$\texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

$\texttt{cool_power} = \texttt{pfl_dop}\times \texttt{MB}\times kv$

In [15]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38064852*1e-23  # Boltzman
    m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def cool_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)*nu

def hot_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [16]:
T = 0.01
v_thermique = sqrt(2*kb*T/m_Ca)
nv = 2000
dv = 6*v_thermique/nv
dist_MB = []
for v in linspace(-3*v_thermique,3*v_thermique,nv):
    dist_MB.append(MB(v, T))
print(sum(dist_MB)*dv)

0.9994781288008786


In [17]:
# Laser parameters
lam = 397e-9 # m
klam = 2*pi/lam # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
Rab = Gam
I = 170 # W/m²

In [18]:
c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
lam_397 = 396.84620*1e-9
k=(2*pi)/lam_397
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
Omega = 2.0e6*2*pi
tau_RF = 1/(Omega/(2*pi))

In [19]:
v_min = sqrt(h_pl_bar*Gam/m_Ca)
print(v_min)
print(m_Ca*v_min**2/kb)

0.4634206649198229
0.0010351970908243904


# Windowing

In [20]:
# Windowing temperature
window_samp = 200
x_win = [] # zeros((len(y)//window_samp,len(file_path)))
y_win = [] # zeros((len(y)//window_samp,len(file_path)))

for to_fit in range(len(file_path)):
    print(to_fit)
    x_win.append([])
    y_win.append([])
    x = time[to_fit]-1e-3
    y = T_aux_avg[to_fit]
    for i in range(len(y)//window_samp-1):
        x_win[to_fit].append( (x[i*window_samp]+x[(i+1)*window_samp])/2 ) # temps
        y_win[to_fit].append( sum(y[i*window_samp:(i+1)*window_samp])/window_samp ) # temperature moyennée
    
print(diff(x_win[0])[0]*1e3,tau_RF*window_samp*1e3)

0
0.10000000000000005 0.09999999999999999


In [21]:
# Plot windowed temperature
to_plot = 0
x  = time[to_plot]-1e-3
y  = T_aux_avg[to_plot]
figure('Windowed T',clear='True')
semilogy(x, y,':',color='y')
semilogy(x_win[to_plot], y_win[to_plot],'+')
# hlines(500,0,0.02)
grid()
tight_layout()

In [24]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

nv = 500
log_temps = linspace(-3.1,3,nv)
coolpow_05 = []
coolpow_1 = []
# coolpow_1_bis = []
coolpow_2 = []
coolpow_4 = []
coolpow_6 = []
# coolpow_8 = []
coolpow_10 = []
# coolpow_50 = []
hotpow = []
hotpow_1 = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 6*v_thermique/nv
    coolpow_05.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_1.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    hotpow.append( sum(hot_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam))*dv*h_pl_bar**2*klam**2/m_Ca*Gam/kb*tau_RF )
    hotpow_1.append( sum(hot_power(-dv*nv/2, dv*nv/2, nv, T, -Gam, klam, Rab, Gam))*dv*h_pl_bar**2*klam**2/m_Ca*Gam/kb*tau_RF )
my_color=[7,2,6,3,3] # 
figname = 'dT_vs_T_g_10g'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [0]
start_lim = [0,0,750,755,20]
end_lim   = [1950,650,75,150,50]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][start_lim[to_plot]:-1-end_lim[to_plot]],
               diff(y_win[to_plot][start_lim[to_plot]:-end_lim[to_plot]])/window_samp, # dT par tau RF
              marker='.',ms=14,mec='k',mew=0.2,ls='',color=cm4((incr+0.3)/4),
              label=f'{q[my_try]}')
    incr+=1

max_H = 50
ax1.loglog(y_win[0][max_H-50:max_H+15-1], diff(y_win[0][max_H-50:max_H+15])/window_samp,
      marker=' ',ls=':',lw=2.5,color=cm4((0+0.3)/4))

doyouwantallthelabels = False
if doyouwantallthelabels == True:
#     ax1.loglog(10**log_temps,coolpow_05,ls="-",color='xkcd:red',lw=4,label='$-0.5\Gamma$')
    ax1.loglog(10**log_temps,coolpow_1,ls="-",marker='+',color='xkcd:red',lw=4,label='$-\Gamma$')
#     ax1.loglog(10**log_temps,coolpow_4,ls="dashdot",color='xkcd:red',lw=4,label='$-4\Gamma$')  # cm3j(3/10)
    ax1.loglog(10**log_temps,coolpow_10,ls="--",color='xkcd:red',lw=4,label='$-10\Gamma$') # cm3j(9/10) (0,(1,1))
#     ax1.loglog(10**log_temps[:nv-50],hotpow[:nv-50],ls='-',color='k',lw=4,label='$H_e(-\Gamma/2)$') # 'dashdot'
    ax1.loglog(10**log_temps[:nv],hotpow_1[:nv],ls='-',color='k',lw=4,label='$H_e(-\Gamma)$') # 'dashdot'
else:
#     ax1.loglog(10**log_temps,coolpow_05,ls="-",color='xkcd:red',lw=4)
    ax1.loglog(10**log_temps,coolpow_1,ls="-",marker='P',color='xkcd:red',lw=4)
#     ax1.loglog(10**log_temps,coolpow_4,ls="dashdot",color='xkcd:red',lw=4)  # cm3j(3/10)
    ax1.loglog(10**log_temps,coolpow_10,ls='--',color='xkcd:red',lw=4) # cm3j(9/10) (0, (3, 5, 1, 5, 1, 5)) (0,(1,1))
#     ax1.loglog(10**log_temps[:nv-158],hotpow[:nv-158],ls='-',color='k',lw=4) # 'dashdot' xkcd:purple
    ax1.loglog(10**log_temps[:nv-150],hotpow_1[:nv-150],ls='-',color='k',lw=4) # 'dashdot'


ax1.set_xlabel('$T$ (K)')
ax1.set_ylabel(r'$\mathrm{d}T/\mathrm{d}t\; (K/\tau_{RF})$')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.legend(title=f'$q_x$',loc=4,ncol=1,fontsize=22)
ax1.set_xlim(7e-3,0.3e3)
ax1.set_ylim(1e-8,1e-1)

tight_layout()

# savefig(figname+'qvar'+'.eps',dpi=300)
# savefig(figname+'qvar'+'.jpg',dpi=300)

K:/Universite_Simus/20230413/Time_and_temp_RFHEAT_N1024_alphavar_DC02_RF02.npz


In [248]:
from scipy import signal

In [26]:
# Find the Temperature for G = H
# First I find, for each H curve, the point just below G
# Then linear interpol between this point and the next
# to refine intersection point
# be careful linear in log


idx = []
heating = []
x = []
x_max = [60, 25, 850, 900, 100]

# Generate G cooling curve
# with the same H heating curve resolution
coolpow_05_interpol = []
coolpow_1_interpol  = []
t0_interpol = []
choosen_plot = [0]
for to_plot in choosen_plot:
    x.append(y_win[to_plot][:x_max[to_plot]])
    coolpow_05_interpol.append([])
    coolpow_1_interpol.append([])
    for T in x[to_plot]:
        v_thermique = sqrt(2*kb*T/m_Ca)
        dv = 6*v_thermique/nv
        coolpow_05_interpol[to_plot].append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
        coolpow_1_interpol[to_plot].append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )

    heating.append(diff(x[to_plot])/window_samp)
    # Find index where both curve crosses
    # in fact the point just before crossing
    idx_temp = np.argwhere(np.diff(np.sign( array( heating[to_plot] )  - array( coolpow_1_interpol[to_plot][:-1] ) ))).flatten()
    try:
        idx.append(idx_temp[0])
    except :
        idx.append(0)


figname = 't0_interpol'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)
incr = 0
# t0_interpol = []
# Going to produce a linear interpolation
# between the two points
# For each heating rate curve
# one point below the cooling
# one point above
# linear interpolation of both G and H
# then find the intersection point

t0_linear_interpol = []
T_interpol = []
for to_plot in choosen_plot:
    ax1.loglog(x[to_plot][:-1], diff(x[to_plot])/window_samp, color=cm4((incr+0.3)/4),marker='o',ls='')
    ax1.loglog(x[to_plot][idx[to_plot]], diff(x[to_plot])[idx[to_plot]]/window_samp, color=cm4((incr+0.3)/4),marker='x',ms=15,ls='')
    ax1.loglog(10**log_temps,coolpow_1,ls="-",marker='P',color='xkcd:red',lw=4)
    
    # create linear interpolations
    x_lin = np.linspace( log10(x[to_plot][idx[to_plot]]),log10(x[to_plot][idx[to_plot]+1]),1000 )
    y_win_lin = np.linspace( log10(diff(x[to_plot])[idx[to_plot]]/window_samp),log10(diff(x[to_plot])[idx[to_plot]+1]/window_samp),1000 )
    coolpow_05_lin = []
    coolpow_1_lin = []
    for T in 10**x_lin:
        v_thermique = sqrt(2*kb*T/m_Ca)
        dv = 6*v_thermique/nv
        coolpow_05_lin.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
        coolpow_1_lin.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )

    idx_lin = np.argwhere(np.diff(np.sign( coolpow_1_lin - 10**y_win_lin ))).flatten()
    print(idx_lin)
    ax1.loglog(10**x_lin[idx_lin],10**y_win_lin[idx_lin], color=cm4((incr+0.3)/4),marker='P',ms=15,ls='')
    
    incr+=1
    try:
        T_interpol.append(x_lin[idx_lin][0])
#         print(idx_lin[0])
        print(x_win[to_plot][idx[to_plot]]*1e3,x_win[to_plot][idx[to_plot]+1]*1e3)
#         print(to_plot)
#        t0_linear_interpol.append()  
    except :
        T_interpol.append(-10)
#         t0_linear_interpol.append() 
    
#     
ax1.set_xlabel('T')
ax1.set_ylabel('dT/dt')
grid()
tight_layout()

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in log10
H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in sign


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24